In [ ]:
import h5py
import matplotlib_inline
import numpy as np
import time

from tqdm import tqdm

from smml.kernels import GaussianKernel, PolynomialKernel
from smml.models import MulticlassPegasos
from smml.cross_validation import (
    KFoldCrossValidation, ParamGridCrossValidation
)
from smml.utils import (
    plot_class_counts, plot_cv_runtime, 
    plot_digits, plot_heatmap,
    plot_runtime_comparison, plot_tsne_data
)

matplotlib_inline.backend_inline.set_matplotlib_formats('retina')

In [ ]:
PATH = '../../datasets/usps/usps.h5'
with h5py.File(PATH, 'r') as hf:
        train = hf.get('train')
        X_tr = train.get('data')[:]
        y_tr = train.get('target')[:]
        test = hf.get('test')
        X_te = test.get('data')[:]
        y_te = test.get('target')[:]

X = np.concatenate((X_tr, X_te))
y = np.concatenate((y_tr, y_te))

# Dataset overview

In [ ]:
plot_digits(X, y, 10)

In [ ]:
plot_class_counts(y)

In [ ]:
plot_tsne_data(X, y)

# Runtime comparison

In [ ]:
def naive_training(X, y, kernel, T, l=0.1, seed=42):
    alphas = np.zeros(X.shape[0])
    rng = np.random.default_rng(seed)
    for t in range(1, T + 1):
        i = rng.integers(X.shape[0])
        s = np.sum(
            [alphas[j] * y[j] * kernel(X[i], X[j])
                for j in range(X.shape[0])])
        if (y[i] / (l * t)) * s < 1:
            alphas[i] += 1

    return alphas

def optimized_training(X, y, kernel, T, l=0.1, seed=42):
    alphas = np.zeros(X.shape[0])
    rng = np.random.default_rng(seed)
    K = kernel.compute_kernel_matrix(X, X)

    for t in range(1, T + 1):
        i = rng.integers(X.shape[0])
        s = (alphas * y).dot(K[i])
        if (y[i] / (l * t)) * s < 1:
            alphas[i] += 1

    return alphas

In [ ]:
np.random.seed(10)
N = 1000
X = np.random.rand(N, 10)
y = np.random.randint(2, size=N)
kernel = GaussianKernel()
Ts = np.linspace(100, 5000, 10, dtype=int)

In [ ]:
times = {
    'naive': [],
    'optimized': []
}

for T in tqdm(Ts, leave=False):
    start = time.process_time()
    res1 = naive_training(X, y, kernel, T)
    times['naive'].append(time.process_time() - start)
    start = time.process_time()
    res2 = optimized_training(X, y, kernel, T)
    times['optimized'].append(time.process_time() - start)
    assert np.allclose(res1, res2)

In [ ]:
plot_runtime_comparison(Ts, times)

# Cross-validation experiments

In [ ]:
cv = KFoldCrossValidation(5)
param_grid = {'T': [1000, 5000, 10000, 25000, 50000], 
              'l': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}

## Polynomial kernel (degree 2)

In [ ]:
mp = MulticlassPegasos(kernel=PolynomialKernel(n=2))
pgcv = ParamGridCrossValidation(mp, param_grid, cv)
res = pgcv.fit(X, y)

In [ ]:
with open(r'results/poly_2.txt','w+') as f:
    f.write(str(res))

In [ ]:
res = ''
with open(r'results/poly_2.txt','r') as f:
     for i in f.readlines():
         res=i
res = eval(res)

plot_heatmap(
     np.array([val['error'] for val in res.values()]).round(3), 
     param_grid, r'\textbf{Polynomial Kernel (degree 2)}', 
     r'Regularization coefficient ($\lambda$)', 'Number of rounds (T)',
     'Test error (zero-one loss)', 'img/poly_2_error.png')

## Polynomial kernel (degree 3)

In [ ]:
mp = MulticlassPegasos(kernel=PolynomialKernel(n=3))
pgcv = ParamGridCrossValidation(mp, param_grid, cv)
res = pgcv.fit(X, y)

In [ ]:
with open(r'results/poly_3.txt','w+') as f:
    f.write(str(res))

In [ ]:
res = ''
with open(r'results/poly_3.txt','r') as f:
     for i in f.readlines():
         res=i
res = eval(res)

plot_heatmap(
     np.array([val['error'] for val in res.values()]).round(3), 
     param_grid, r'\textbf{Polynomial Kernel (degree 3)}', 
     r'Regularization coefficient ($\lambda$)', 'Number of rounds (T)',
     'Test error (zero-one loss)', 'img/poly_3_error.png')

## Polynomial kernel (degree 4)

In [ ]:
mp = MulticlassPegasos(kernel=PolynomialKernel(n=4))
pgcv = ParamGridCrossValidation(mp, param_grid, cv)
res = pgcv.fit(X, y)

In [ ]:
with open(r'results/poly_4.txt','w+') as f:
    f.write(str(res))

In [ ]:
res = ''
with open(r'results/poly_4.txt','r') as f:
     for i in f.readlines():
         res=i
res = eval(res)

plot_heatmap(
     np.array([val['error'] for val in res.values()]).round(3), 
     param_grid, r'\textbf{Polynomial Kernel (degree 4)}', 
     r'Regularization coefficient ($\lambda$)', 'Number of rounds (T)',
     'Test error (zero-one loss)', 'img/poly_4_error.png')

## Polynomial kernel (degree 7)

In [ ]:
mp = MulticlassPegasos(kernel=PolynomialKernel(n=7))
pgcv = ParamGridCrossValidation(mp, param_grid, cv)
res = pgcv.fit(X, y)

In [ ]:
with open(r'results/poly_7.txt','w+') as f:
    f.write(str(res))

In [ ]:
res = ''
with open(r'results/poly_7.txt','r') as f:
     for i in f.readlines():
         res=i
res = eval(res)

plot_heatmap(
     np.array([val['error'] for val in res.values()]).round(3), 
     param_grid, r'\textbf{Polynomial Kernel (degree 7)}', 
     r'Regularization coefficient ($\lambda$)', 'Number of rounds (T)',
     'Test error (zero-one loss)', 'img/poly_7_error.png')

## Gaussian kernel (gamma 0.25)

In [ ]:
mp = MulticlassPegasos(kernel=GaussianKernel(0.25))
pgcv = ParamGridCrossValidation(mp, param_grid, cv)
res = pgcv.fit(X, y)

In [ ]:
with open(r'results/gaussian_25.txt','w+') as f:
    f.write(str(res))

In [ ]:
res = ''
with open(r'results/gaussian_25.txt','r') as f:
     for i in f.readlines():
         res=i
res = eval(res)

plot_heatmap(
     np.array([val['error'] for val in res.values()]).round(3), 
     param_grid, r'\textbf{Gaussian Kernel ($\gamma$ 0.25)}', 
     r'Regularization coefficient ($\lambda$)', 'Number of rounds (T)',
     'Test error (zero-one loss)', 'img/gaussian_25_error.png')

## Gaussian Kernel (gamma 0.75)

In [ ]:
mp = MulticlassPegasos(kernel=GaussianKernel(0.75))
pgcv = ParamGridCrossValidation(mp, param_grid, cv)
res = pgcv.fit(X, y)

In [ ]:
with open(r'results/gaussian_75.txt','w+') as f:
    f.write(str(res))

In [ ]:
res = ''
with open(r'results/gaussian_75.txt','r') as f:
     for i in f.readlines():
         res=i
res = eval(res)

plot_heatmap(
     np.array([val['error'] for val in res.values()]).round(3), 
     param_grid, r'\textbf{Gaussian Kernel ($\gamma$ 0.75)}', 
     r'Regularization coefficient ($\lambda$)', 'Number of rounds (T)',
     'Test error (zero-one loss)', 'img/gaussian_75_error.png')

## Gaussian Kernel (gamma 2)

In [ ]:
mp = MulticlassPegasos(kernel=GaussianKernel(2))
pgcv = ParamGridCrossValidation(mp, param_grid, cv)
res = pgcv.fit(X, y)

In [ ]:
with open(r'results/gaussian_2.txt','w+') as f:
    f.write(str(res))

In [ ]:
res = ''
with open(r'results/gaussian_2.txt','r') as f:
     for i in f.readlines():
         res=i
res = eval(res)

plot_heatmap(
     np.array([val['error'] for val in res.values()]).round(3), 
     param_grid, r'\textbf{Gaussian Kernel ($\gamma$ 2)}', 
     r'Regularization coefficient ($\lambda$)', 'Number of rounds (T)',
     'Test error (zero-one loss)', 'img/gaussian_2_error.png')

## Average cross-validation runtime

In [ ]:
files = ['results/poly_2.txt', 'results/poly_3.txt', 
         'results/poly_4.txt', 'results/poly_7.txt', 
         'results/gaussian_25.txt', 'results/gaussian_75.txt', 
         'results/gaussian_2.txt']
labels = ['polynomial (degree 2)', 'polynomial (degree 3)', 
          'polynomial (degree 4)', 'polynomial (degree 7)',
          'gaussian (gamma 0.25)', 'gaussian (gamma 0.75)',
          'gaussian (gamma 2)']

plot_cv_runtime(files, labels, param_grid)